# Vectormorph Projet 3A Résultats
Luther OLLIER, Stanislas MARESCHAL DE CHARENTENAY, Herve SILUE
<br/>
## Sommaire :
- Contexte et Problématique
- Les Datasets
- Training sur les images
- Training sur les images transformées
- Comparaison des résultats


## Contexte et Problématique

Le recalage en imagerie médicale est une tâche qui consiste à déterminer une représentation spatiale optimale permettant de
superposer le plus exactement deux images (scanner, IRM, TEP, ultrason...).

Cependant cette tâche est rendu difficile par :

- La variabilité des représentations sous-jacentes (mouvement respiratoire, l’évolution d’une pathologie).

- Des difficultés d’acquisition propres à chaque modalité (niveau de bruit important en imagerie ultrasonore, la non-correspondance entre régions fonctionnelles et anatomiques en TEP-scanner).

Le recalage de représentation structurelles (transformations particulières conservant l’information sur la structure) des images donne des résultats équivalents, voir plus intéressants dans certains cas, que le recalage d’images brutes.

On s'intéresse à une représentation structurelle obtenue par la méthode Vector Field Convolution (VFC). Cette transformation consiste à convoluer un noyau vectoriel avec les contours de l’image. La VFC a pour paramètres (a,r) avec r la taille du kernel et a qui caractérise l’évolution de l’amplitude du champ. 

Le but de ce projet est d’étudier les performance du framework Voxelmorph, spécialisé en recalage d’image par deep-learning, sur les représentations structurelles par VFC. 

## Datasets
  
Le dataset initial était composé de 400 IRM du cerveau et des masques de segmentations associés (indispensable au calcul de la métrique Dice choisie comme métrique de performance). Cependant pour des raisons calculatoires (impossible d'entraîner dans des conditions satisfaisantes des scanners 3D de vecteurs) nous nous sommes rabattus sur un Dataset 2D plus simple, le [HandMnist](https://github.com/Project-MONAI/MONAI/blob/master/examples/notebooks/mednist_tutorial.ipynb) 
de Monai contenant 10000 radios des mains. Cependant sans masques de segmentations, nous n'avons pas pu évaluer les performances aussi précisément que nous l'aurions voulu.

In [ ]:
from Dataloaders.MRI_loader import get_labels_list
from PIL import Image
from numpy import asarray
import matplotlib.pyplot as plt


dataset_path=''
images_path=get_labels_list(dataset_path)

fig, axes = plt.subplots(5, 2, figsize=(20,10))
for i in range(10):
    image=asarray(Image.open(images_path[i]))
    axes[i,i-((i//5)*5)].imshow(slice.T, origin="lower")

Pour effectuer la transformation VFC sur les images nous allons convoluer un noyau vectoriel avec les contours de l'image. Ils sont affichés ci-dessous notamment pour les 2 valeurs de a que nous allons utiliser pour les entrainements

In [ ]:
from process.utils import to_vector, image_contour, VFK
import numpy as np

plt.figure(figsize=(15,10))

plt.subplot(221)
plt.title("Raw image")
plt.imshow(image)

plt.subplot(222)
plt.title("Image contour")
plt.imshow(image_contour(fixed_image))

plt.subplot(223)
plt.title("Kernel for a=1.5 and r=10")
K = VFK(a=1.5,r=10)
nkx,nky = K.shape
xv ,yv = np.meshgrid(range(nkx),range(nky))
plt.quiver(xv, yv)

plt.subplot(224)
plt.title("Kernel for a=3 and r=10")
K = VFK(a=3,r=10)
nkx,nky= K.shape
xv ,yv = np.meshgrid(range(nkx),range(nky))
plt.quiver(xv, yv)

Affichons maintenant des images vectorisées :

In [ ]:
plt.figure(5, 3, figsize=(20,10))

nx,ny = fixed_image.shape
xv ,yv = np.meshgrid(range(ny),range(nx))

for i in range(5):
    image=asarray(Image.open(images_path[i]))
    
    plt.subplot(5,3,(5*i))
    plt.title("image")
    plt.imshow(image.T, origin="lower")
    
    plt.subplot(5,3,(5*(i+1)))
    plt.title("VFC for a = 1.5")
    vector_field = to_vector(fixed_image,r=10,a=1.5)
    plt.quiver(xv,yv,vector_field[:,:,0],vector_field[:,:,1])
    
    plt.subplot(5,3,(5*(i+2)))
    plt.title("VFC for a = 3")
    vector_field = to_vector(fixed_image,r=10,a=3)
    plt.quiver(xv,yv,vector_field[:,:,0],vector_field[:,:,1])
    
plt.show()

le coefficient __a__ caractérise l’amplitude du champ vectoriel $k(x,y) =  \frac{v(x,y)}{(r^(a+1)+\epsilon)}$, on observe sur les images que les transformations VFC pour a=3 sont caractérisées par un champ vectoriel plus marqué, moins lisse.

## Training sur les images

Pour une batch size de 32, un dataset d'entrainement de 6000 images et de 4000 pour la validation, voici la courbe d'apprentissage pour 500 epoques :

In [ ]:
from process.utils import loss_recovery

path_output_2D=""
loss_recovery(path_output_2D)


On peut voir que la loss $(MSE+0.05\Delta(vectorfield))$  converge au bout de ** epoch vers **

In [ ]:
from main import evaluate
from config import model_config, dataloader_config, train_config, criterion_config, scheduler_config
model_state_dict_path="path"
trained_model = evaluate(model_state_dict_path, model_config, dataloader_config, train_config, criterion_config, scheduler_config)

In [ ]:
#loading two scan for visualization
import torch

fixed_image,moving_image=next(iter(trainer.testloader))
fixed_image, moving_image = fixed_image.to(trainer.model.device), moving_image.to(trainer.model.device)

with torch.no_grad():
            pred_image, flow_field = trainer.model.net(moving_image.float(),fixed_image.float())
            loss = trainer.model.criterion(pred_image, fixed_image, flow_field)
            
fixed_image,moving_image=fixed_image.detach().cpu().numpy(),moving_image.detach().cpu().numpy()
pred_image, flow_field = pred_image.detach().cpu().numpy(), flow_field.detach().cpu().numpy()

plt.figure(figsize=(12, 6))

plt.subplot(1, 4, 1)
plt.imshow(fixed_image)
plt.title("image cible")

plt.subplot(1, 4, 2)
plt.imshow(moving_image)
plt.title("image à recaler")

plt.subplot(1, 4, 3)
plt.imshow(pred_image)
plt.title("image recalée")

plt.subplot(1, 4, 4)
plt.quiver(xv,yv,flow_field[:,:,0],flow_field[:,:,1])
plt.title("champ de transformation")

plt.show()

blabla

## Training sur les images transformées

Nous avons effectué 2 entrainements, un pour a=1.5 et un autre pour a=3 (amplitude plus forte du champ vectoriel)

### a=1.5
courbe de loss :

In [ ]:
from process.utils import loss_recovery

path_output_2D_vec_0=""
loss_recovery(path_output_2D_vec_0)

On peut voir que la loss converge au bout de ** epoch vers **

In [ ]:
dataloader_config['vectorize']=True
dataloader_config['a']=1.5
model_config['src_feats']=3
model_config['trg_feats']=3
model_state_dict_path="path"
trained_model = evaluate(model_state_dict_path, model_config, dataloader_config, train_config, criterion_config, scheduler_config)

In [ ]:
#loading two scan for visualization
import torch

fixed_image,moving_image=next(iter(trainer.testloader))
fixed_image, moving_image = fixed_image.to(trainer.model.device), moving_image.to(trainer.model.device)

with torch.no_grad():
            pred_image, flow_field = trainer.model.net(moving_image.float(),fixed_image.float())
            loss = trainer.model.criterion(pred_image, fixed_image, flow_field)
            
fixed_image,moving_image=fixed_image.detach().cpu().numpy(),moving_image.detach().cpu().numpy()
pred_image, flow_field = pred_image.detach().cpu().numpy(), flow_field.detach().cpu().numpy()

plt.figure(figsize=(12, 6))

plt.subplot(1, 4, 1)
plt.imshow(fixed_image)
plt.title("image cible")

plt.subplot(1, 4, 2)
plt.imshow(moving_image)
plt.title("image à recaler")

plt.subplot(1, 4, 3)
plt.imshow(pred_image)
plt.title("image recalée")

plt.subplot(1, 4, 4)
plt.quiver(xv,yv,flow_field[:,:,0],flow_field[:,:,1])
plt.title("champ de transformation")

plt.show()

### a=3

courbe de loss :

In [ ]:
from process.utils import loss_recovery

path_output_2D_vec_1=""
loss_recovery(path_output_2D_vec_1)

On peut voir que la loss converge au bout de ** epoch vers **

In [ ]:
dataloader_config['a']=3
model_state_dict_path="path"
trained_model = evaluate(model_state_dict_path, model_config, dataloader_config, train_config, criterion_config, scheduler_config)

In [ ]:
#loading two scan for visualization
import torch

fixed_image,moving_image=next(iter(trainer.testloader))
fixed_image, moving_image = fixed_image.to(trainer.model.device), moving_image.to(trainer.model.device)

with torch.no_grad():
            pred_image, flow_field = trainer.model.net(moving_image.float(),fixed_image.float())
            loss = trainer.model.criterion(pred_image, fixed_image, flow_field)
            
fixed_image,moving_image=fixed_image.detach().cpu().numpy(),moving_image.detach().cpu().numpy()
pred_image, flow_field = pred_image.detach().cpu().numpy(), flow_field.detach().cpu().numpy()

plt.figure(figsize=(12, 6))

plt.subplot(1, 4, 1)
plt.imshow(fixed_image)
plt.title("image cible")

plt.subplot(1, 4, 2)
plt.imshow(moving_image)
plt.title("image à recaler")

plt.subplot(1, 4, 3)
plt.imshow(pred_image)
plt.title("image recalée")

plt.subplot(1, 4, 4)
plt.quiver(xv,yv,flow_field[:,:,0],flow_field[:,:,1])
plt.title("champ de transformation")

plt.show()